In [170]:
import pandas as pd
import gcsfs
from google.cloud import storage
import pyarrow
import os
from google.oauth2 import service_account
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import requests
import json
import pandas as pd
import math
import datetime
from google.cloud import storage,bigquery
from flask import jsonify
import os
# from dotenv import load_dotenv ## for local debugging, comment out when deploy to gcf

# load_dotenv() ## for local debugging, comment out when deploy to gcf


def transform_centanet_data(event,context):
    """Triggered by a change to a Cloud Storage bucket.
        Args:
            event (dict): Event payload.
            context (google.cloud.functions.Context): Metadata for the event.
        """
    # Initialize GCF API Response
    output_json = {}
    
    #Initialize functions

    def load_data_to_table(project_name: str, dataset_name: str, table_name: str, dataframe_clean: pd.DataFrame) -> str:
    ##Directly loads data to table
    # Construct a BigQuery client object.
        client = bigquery.Client()

        table_id = ".".join(project_name,dataset_name,table_name)

        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.PARQUET,
        )

        load_job = client.load_table_from_dataframe(
            dataframe= dataframe_clean, destination=table_id, job_config=job_config
        )  # Make an API request.
        
        result = load_job.result()  # Waits for the job to complete.

        return result


    # Initialize variables
    bucket_name = "project-diver" #event['bucket']
    blob_name = "STAGING/centanet/2022/centanet_property_transaction_111022.parquet"#event['name']
    directory_path = "gs://" + bucket_name + "/"
    source_file_path = directory_path + blob_name
    schema_file_path = directory_path + os.environ.get("SCHEMA_FILE_NAME")

    var_year = str(datetime.datetime.utcnow().strftime('%Y'))
    var_datetime = str(datetime.datetime.utcnow().strftime('%d%m%y'))
    destination_file_path = "./TRANSFORMED/centanent/{0}/TRANSFORMED_centanet_2022_centanet_property_transaction_{1}.csv".format(
        var_year, var_datetime)

    df_raw_property_transaction = pd.read_parquet(source_file_path)

    # schema configuration
    df_schema = pd.read_csv(schema_file_path)[
        ["Source Column", "Target Column"]].dropna(axis=0)

    ## putting the columns into a list - used for extracting 
    list_of_source_columns = df_schema["Source Column"].to_list()


    dict_column_mapping = dict(
        zip(df_schema["Source Column"], df_schema["Target Column"]))

    df_intermediate = pd.concat([df_raw_property_transaction.drop(['picture', 'scope'], axis=1), df_raw_property_transaction["picture"].apply(
        pd.Series), df_raw_property_transaction["scope"].apply(pd.Series)], axis=1)[list_of_source_columns]

    df_intermediate = df_intermediate.explode("floorPlans")
    df_intermediate = df_intermediate.replace({'postType': {'R': 'Rent',
                                                            'S': 'Sale'},
                                               'unitType': {'AP': 'Apartment',
                                                            'HS': 'House'}})

    df_clean = df_intermediate.rename(columns=dict_column_mapping)

    

    return jsonify(output_json)


In [171]:
# Initialize GCF API Response
output_json = {}

#Initialize functions

def load_data_to_table(project_name: str, dataset_name: str, table_name: str, dataframe_clean: pd.DataFrame) -> str:
##Directly loads data to table##
# Construct a BigQuery client object.
    client = bigquery.Client()

    table_id = ".".join([dataset_name,table_name])

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
    )

    load_job = client.load_table_from_dataframe(
        dataframe= dataframe_clean, destination=table_id, job_config=job_config
    )  # Make an API request.
    
    result = load_job.result()  # Waits for the job to complete.

    return result
    
def query_data_from_table(project_name: str, dataset_name: str, table_name: str, query_string: str):
    client = bigquery.Client()
    query_job = client.query(
        query=query_string,
    )  # API Request

    result = query_job.to_dataframe()# Make the request

    return result

# Initialize variables
bucket_name = "project-diver"
blob_name = "STAGING/centanet/2022/centanet_property_transaction_111022.parquet"
directory_path = "gs://" + bucket_name + "/"
source_file_path = "./DATASETS/STAGING_centanet_2022_centanet_property_transaction_111022.parquet"#directory_path + blob_name
schema_file_path = "../MODELS/TABLES/property-transaction-table-structure.csv"#directory_path +"SCHEMAS/property-transaction-table-structure.csv"
latest_inserted_date = str(query_data_from_table("project-diver","project-diver","dbo.property_transaction","SELECT Inserted_Date_Api FROM dbo.property_transaction ORDER BY Inserted_Date_Api DESC LIMIT 1")["Inserted_Date_Api"][0])

var_year = str(datetime.datetime.utcnow().strftime('%Y'))
var_datetime = str(datetime.datetime.utcnow().strftime('%d%m%y'))
destination_file_path = "./TRANSFORMED/centanent/{0}/TRANSFORMED_centanet_2022_centanet_property_transaction_{1}.csv".format(
    var_year, var_datetime)


In [172]:

## Read data
df_raw_property_transaction = pd.read_parquet(source_file_path)

## read schema configuration
df_schema = pd.read_csv(schema_file_path)[
    ["Source Column", "Target Column"]].dropna(axis=0)


In [174]:

## putting column names into a list - used for extracting 
list_of_source_columns = df_schema["Source Column"].to_list()
dict_column_mapping = dict(
    zip(df_schema["Source Column"], df_schema["Target Column"]))

## cleaning data
df_intermediate = pd.concat([df_raw_property_transaction.drop(['picture', 'scope'], axis=1), df_raw_property_transaction["picture"].apply(
    pd.Series), df_raw_property_transaction["scope"].apply(pd.Series)], axis=1)[list_of_source_columns]
df_intermediate['insDate'] = pd.to_datetime(df_intermediate['insDate'])
df_intermediate = df_intermediate[(df_intermediate['insDate']>datetime.datetime.strptime(latest_inserted_date,"%Y-%m-%d %H:%M:%S"))]

df_intermediate = df_intermediate.explode("floorPlans")
df_intermediate = df_intermediate.replace({'postType': {'R': 'Rent',
                                                        'S': 'Sale'},
                                            'unitType': {'AP': 'Apartment',
                                                        'HS': 'House'}})

df_clean = df_intermediate.rename(columns=dict_column_mapping)
load_data_to_table("project-diver","project-diver","dbo.property_transaction",df_clean)

LoadJob<project=project-diver, location=US, id=ad76e8d9-cdd9-4446-b121-f83bc64109c3>

In [175]:
client = bigquery.Client()

table_id = ".".join(["project-diver","dbo.property_transaction"])

load_job = client.load_table_from_dataframe(
    dataframe= df_clean, destination=table_id
)  # Make an API request.

result = load_job.result()

In [169]:
print(result.output_rows)

131
